This Notebook solves the 2site hexagonal lattice hubbard model using NNgHMC method

The hamiltonian required for 2-site Hubbard model is :
\begin{equation}
    \begin{split}
    H & = \frac{\vec{p}^2}{2} + \frac{\vec{\phi}^2}{2\tilde{U}} - \log\det \left[M[\phi,\tilde{\kappa},\tilde{\mu}]M[-\phi,-\tilde{\kappa},-\tilde{\mu}]\right] \\ & \equiv  \frac{\vec{p}^2}{2} + S_{eff}[\phi], 
    \end{split}
\end{equation}
where the effective action $S_{eff}$ comes from the Hubbard-Stratonovich transformation: 
\begin{equation}
    S_{eff}[\phi] = \frac{\vec{\phi}^2}{2\tilde{U}} - \log\det \left[M[\phi,\tilde{\kappa},\tilde{\mu}]M[-\phi,-\tilde{\kappa},-\tilde{\mu}]\right].
\end{equation}
and the gradient of $S_{eff}$ is what the NN approximates:
    
\begin{equation}
\partial_{\phi_{xt}}S_{eff}[\phi]=\frac{\phi_{xt}}{U\delta}-\partial_{\phi_{xt}}\log\det M[\phi]M[-\phi] =\frac{\phi_{xt}}{U\delta}-2\operatorname{Re} \operatorname{tr}\left(M^{-1}[\phi]\partial_{\phi_{xt}}M[\phi]\right)
\end{equation}

Here the parameters of the Hubbard model, the potential energy $U$, the chemical potential $\mu$, and the hopping matrix $\kappa$ are scaled to the discretized time $\delta$:

\begin{equation}
    \delta =\frac{\beta}{N_t}, \quad \tilde{\kappa}=\delta\kappa, \quad \tilde{U}=\delta U, \quad \tilde{\mu} = \delta\mu.
\end{equation}

The elements of the fermion matrix $M[\phi,\tilde{\kappa},\tilde{\mu}]$ can be calculated by *exponential discretization*:
    
\begin{equation}
    M[\phi,\tilde{\kappa},\tilde{\mu}]_{x't';xt} =\delta_{x',x}\delta _{t',t}-B_{t'}\left[e^{h-\tilde{\mu}}\right]_{x',x}e^{i\phi_{x,t'-1}}\delta_{t',t+1},
\end{equation}
                                                                                                                            
with the hopping matrix $h_{x',x}=\tilde{\kappa}\delta_{\langle x',x\rangle}$, and the anti-periodic boundary condition:
                                                                                                                            
\begin{equation}
    B_{t} =\begin{cases}
    +1, & 0<t<N_t \\
    -1, & t =0.
  \end{cases}
\end{equation}
    

In [1]:
#import the modules 
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from scipy import special
import scipy.stats as stats
from numba import jit
from tqdm.notebook import tqdm


# Functions required

To get the force equations, we need to calculate the following:
​
\begin{equation}
\partial_{\phi_{xt}}\log\det M[\phi]M[-\phi] = \operatorname{tr}\left(M^{-1}[\phi]\partial_{\phi_{xt}}M[\phi]\right)+ \operatorname{tr}\left(M^{-1}[-\phi]\partial_{\phi_{xt}}M[-\phi]\right)
\end{equation}
​
For this particular problem, since it is bi-partite, half-filling, and we are working in the particle/hole basis, the above equation simplifies to
​
\begin{equation}
\partial_{\phi_{xt}}\log\det M[\phi]M[-\phi] =2\operatorname{Re} \operatorname{tr}\left(M^{-1}[\phi]\partial_{\phi_{xt}}M[\phi]\right)
\end{equation}
​



In [2]:
def calcExpK(delta):
    """Function that calculates exponential of hopping matrix"""
    return [[np.cosh(delta),np.sinh(delta)],[np.sinh(delta),np.cosh(delta)]]

def Mphi(phi, expk, nt):
    """
    Function that updates the Fermion matrix
    
    INPUT:
            phi - phi field (array)
            k - hopping parameter (array)
            nt - number of time steps
                        
    OUTPUT:
            M - fermion matrix (update M) 
            
            This uses space index fastest (like Isle)
    """
    n = len(phi)
    nx = int(n/nt)
    if n == M.shape[0]: # this tests if the phi array has the right dimensions
        for t in range(nt-1): # loop over time bloks
            for x in range(nx): # loop over cords and kappa matrix
                M[t*nx+x][t*nx+x] = 1.0 + 0j # diagonal term
                for y in range(nx): # run over the kappa matrix
                    M[(t+1)*nx+x][t*nx+y] = -expk[x][y]*np.exp(1j*phi[t*nx+y]) # off-diagonal
        for x in range(nx): 
            M[(nt-1)*nx+x][(nt-1)*nx+x] = 1.0+0j # diagonal term
            for y in range(nx): # anti-periodic boundary condition
                M[x][(nt-1)*nx+y] = expk[x][y]*np.exp(1j*phi[(nt-1)*nx+y])
        return 0
    else:
        print('# Error! phi and M have inconsistent dimensions!')
        return -1
    
def calcLogDetMM(phi, expk, nt):
    """
    Function that calculates the determinant of Fermion matrices
    
    INPUT:
            phi - phi field (array)
            expk - exponential of hopping parameter (array)
            Nt - number of time steps
                        
    OUTPUT:
            detMM - determinant of the matrices
    """
    
    Mphi(phi, expk, nt) # update M with +1 phi
    detMM = np.log(np.linalg.det(M)) # calc detM with +1 phi
    Mphi(-np.array(phi), expk, nt) # update M with -1 phi
    detMM += np.log(np.linalg.det(M)) # calc detM with -1 phi
    return detMM

def calcTrMM(phi, expk, nt, sign):
    """
    Function that calculates the trace of a Fermion matrix using exponential discretization
    
    INPUT:
            phi - phi field (array)
            expk - exponential of hopping term (array)
            nt - number of time steps
            sign - sign of phi (+1 / -1)
                        
    OUTPUT:
            TrMM - trace of the matrix
    """
    TrMM = [] # trace container
    n = len(phi)
    nx = int(n/nt)
    Mphi(phi, expk, nt) # update M
    invM = np.linalg.inv(M)  # only need to invert once!
    for t in range(nt-1): # loop over time blocks
        for x in range(nx): # loop over sites  (space is fastest)
            temp = 0 + 0j
            for y in range(nx):
                temp += invM[t*nx+x][(t+1)*nx+y]*expk[y][x]
            TrMM.append(temp*(-sign*1j*np.exp(1j*phi[t*nx+x])))
    for x in range(nx): # anti-periodic boundry conditions
        temp = 0 + 0j
        for y in range(nx):
            temp += invM[(nt-1)*nx+x][y]*expk[y][x]
        TrMM.append(temp*sign*1j*np.exp(1j*phi[(nt-1)*nx+x]))

    return np.array(TrMM)

def artH(p, phi, expk, nt, U):
    """
    Function that calculates the artificial Hamiltonian of the Hubbard model
    
    INPUT:
            p - conjugate momentum (array)
            phi - phi field (array)            
            expk - exponential of hopping array (array)
            Nt - number of timesteps
            U - onsite coupling (reduced quantity = U * delta)
            
            (optional)
            beta = inverse temperature
                        
    OUTPUT:
            H - artificial Hamiltonian
    """
        
    H = .5*(np.array(p)@np.array(p)+np.array(phi)@np.array(phi)/U) 
    
    H -= np.real(calcLogDetMM(phi, expk, nt))
    return H



def gradS(phi,U,Nt):
    """function that calculates gradient"""
    return phi/U - 2*np.real(calcTrMM(phi,expk,Nt,1))

def ribbit(p, phi, expk, nt, U, Nmd,eps, surrogate, trajLength = 1.):
    """
    Molecular dynamics integrator (Leap frog algorithm)
    
    INPUT:
            p - conjugate momentum (array)
            phi - phi field (array)
            expk - exponential of hopping parameter (array)
            nt - number of timesteps
            Nmd - number of trajectory pieces
            U - onsite coupling
            eps - integration step
            surrogate - NN trained gradient
            (optional)
            trajLength - length of trajectory
            
            
    OUTPUT:
            (p, phi) - after integration
    """    

 
    p = np.array([p[i] for i in range(len(p))])
    phi = np.array([phi[i] for i in range(len(phi))])
    

    
    phi += 0.5*eps*p # first half step
    
    # steps of integration
    #L = int(np.random.uniform(0, 1)* Nmd)
    for _ in range(Nmd-1):
        if surrogate is None:
            p -= eps*gradS(phi,Udelta,nt)
        else:            
            p -= eps*surrogate.gradient(phi)          #<====== replaced with NN
        phi += eps*p

    # last half step
    if surrogate is None:
        p -= eps*gradS(phi,Udelta,nt)      
    else:
        p -= eps*surrogate.gradient(phi)             #<====== replaced with NN
    phi += 0.5*eps*p
    
    return p, phi




In [3]:

def HMC(Nmd,eps,surrogate):
    """ 
    Function performs the HMC
    
    INPUT:
    Nmd : Number of molecular dynamic steps
    eps : integration step
    surrogate : None for Normal else NNgHMC
    
    
    OUTPUT:
    ensemble : final phi's
    prob : acceptance probability
    
    """
    prob = [] # stores probability
    ensemble = []  # store the individual configurations here

    # sample phi from normal distribution with sigma = sqrt(u)
    phi = np.array([np.random.normal(0,usqrt) for i in range(Nt*Nx)])

    for traj in tqdm(range(nTrajs)):
        # sample momentum from normal distribution w/ sigma = 1
        initP = np.array([np.random.normal(0,1) for i in range(Nt*Nx)])

        initPhi = phi

        initH = artH(initP, initPhi, expk, Nt, Udelta) # initial Hamiltonian

        finP, finPhi = ribbit(initP, initPhi, expk, Nt, Udelta, Nmd,eps,surrogate)

        finH = artH(finP, finPhi, expk, Nt, Udelta) # final hamiltonian

        # accept/reject step
        if np.random.uniform(0,1) <= np.exp(-np.real(finH-initH)): # accept
            phi = finPhi
            ensemble.append(phi)
            prob.append(1.)
        else: # reject
            ensemble.append(phi)
            prob.append(0.)

    prob = np.array(prob)

    return ensemble, prob

# Constants for the functions

In [4]:
# initialize constants for the functions
U=2. # spin coupling
beta=4. # inverse temperature
Nt=16 # number of time steps
Nx = 2 # number of sites
delta = beta/Nt # discretised time
Udelta = delta*U  #reduced U
usqrt = np.sqrt(Udelta) 
M = np.identity(Nt*Nx) + 0j # (Nt*Ni) x (Nt*Ni) identity matrix
expk = calcExpK(delta) # calc exp(kappa)
#Nmd = 10# 3-5 for best acceptance >= 70%
nTrajs = 10000 # number of auxiliary fields in the ensemble

# Actual HMC

In [ ]:
actual_HMC,actualHMC_prob = HMC(Nmd=5,eps=1.0/5.0,surrogate = None)
print("actual HMC prob",actualHMC_prob.mean())

# Creating Training data

In [ ]:
# Generate training data from radnom sampling of normal distribution
num_samples = 10000
training_gaus = np.zeros((num_samples,Nt*Nx))
gradient_gaus = np.zeros((num_samples,Nt*Nx))

data_actual = np.array(actual_HMC)
gradient_actual = np.zeros((len(actual_HMC),Nt*Nx))

#stores data from Actual HMCdatata
xx = np.zeros((len(actual_HMC)+ num_samples,Nt*Nx))
yy= np.zeros((len(actual_HMC)+ num_samples,Nt*Nx))

for i in range(num_samples): 
    training_gaus[i,:] = np.random.normal(0,usqrt,Nt*Nx)
    gradient_gaus[i,:] = gradS(training_gaus[i,:],U*delta,Nt)
    gradient_actual[i,:] = gradS(data_actual[i,:],U*delta,Nt)
    


xx[:num_samples,:] = training_gaus[:num_samples,:]
xx[num_samples:,:] = data_actual[:,:]

yy[:num_samples,:] = gradient_gaus[:num_samples,:]
yy[num_samples:,:] = gradient_actual[:,:]


plt.hist(data_actual.flatten(),density=True,bins=30,label='training_gaus')
plt.legend()



In [ ]:
plt.hist(gradient_actual.flatten(),density=True,bins='auto',label='HMC')
plt.legend()

# Data preparation fot training NN in tensorflow

In [ ]:
# from sklearn import preprocessing
# from sklearn.model_selection import train_test_split

# x = xx
# y = yy

# scaled_data = preprocessing.StandardScaler().fit(x) #scaling the data to zero mean and unit variance
# x = preprocessing.scale(x)
# X_train, X_test, y_train, y_test = train_test_split(x,y, test_size=0.20)

# trial_data = np.zeros((num_samples,Nt*Nx)) # data for testing the NN
# for i in range(num_samples): 
#     trial_data[i,:] = np.random.normal(0,usqrt,Nt*Nx)
    
# trial_scaled = preprocessing.StandardScaler().fit(trial_data)

 # Train the NN to approximate gradient with Tensorflow

In [ ]:
# # Train the NN to approximate gradient
# %reload_ext autoreload
# %autoreload 2
# from models import exper, build

# epoch = 80
# batch = 150
# model = exper(32, [64] , 32)
# #model = build(32,[64],[32])

# r = model.fit(x,y,epochs=epoch, batch_size=batch,verbose=2)

# #r = model.fit(X_train, y_train,validation_data=(X_test,y_test),epochs=epoch, batch_size=batch,verbose=2)


# loss, accuracy = model.evaluate(X_test, y_test)
# print('Accuracy: %.2f'% (accuracy*100), 'Loss: %.2f' % (loss))



In [ ]:
# #Using the NN inside HMC to sample points
# from surrogate import NeuralGrad
# grad_hat = NeuralGrad(model,trial_scaled)
# NNg ,NNg_prob = HMC(surrogate = grad_hat, training = False)
# print("actualHMC_prob=",actualHMC_prob.mean())
# print("NNg_prob=",NNg_prob.mean())

# Train the NN to approximate gradient with Pytorch

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
from sklearn.metrics import accuracy_score
from itertools import chain
from tqdm.notebook import tqdm

class NNg(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        """
        Args:
        input_dim: int for input dimension
        hidden_dim: list of int for multiple hidden layers
        output_dim: list of int for multiple output layers
        """

        # calling constructor of parent class
        super().__init__()

        # defining the inputs to the first hidden layer
        self.hid1 = nn.Linear(input_dim, hidden_dim[0])
        nn.init.normal_(self.hid1.weight, mean=0.0, std=0.01)
        nn.init.zeros_(self.hid1.bias)
        
        self.act1 = nn.ReLU()
#         self.act1 = nn.SiLU()

        # defining the inputs to the output layer
        self.hid2 = nn.Linear(hidden_dim[0], output_dim)
        nn.init.xavier_uniform_(self.hid2.weight)

    def forward(self, X):

        # input and act for layer 1
        X = self.hid1(X)
        X = self.act1(X)

        # input and act for layer 2
        X = self.hid2(X)
        return X

epochs = 180
batchsize = 150
#to access a small section of the training data using the array indexing
inputs,targets = torch.from_numpy(xx).float(),torch.from_numpy(yy).float()
train_ds = TensorDataset(inputs, targets)
# split the data into batches
train_dl = DataLoader(train_ds, batchsize,shuffle=False)
test_dl = DataLoader(train_ds, batchsize, shuffle= False)
model_torch = NNg(Nt*Nx,[2*Nt*Nx],Nt*Nx)
optimizer = torch.optim.Adam(model_torch.parameters(),weight_decay=1e-5)
# optimizer = torch.optim.SGD(model_torch.parameters(),lr=0.01,weight_decay=1e-5)
criterion = nn.L1Loss()

train_losses =[]

# iterate through all the epoch
for epoch in tqdm(range(epochs)):
    running_loss=0
    # go through all the batches generated by dataloader
    for i, (inputs, targets) in enumerate(train_dl):
            # clear the gradients
            optimizer.zero_grad()
            # compute the model output
            yhat = model_torch(inputs)
            # calculate loss
            loss = criterion(yhat, targets.type(torch.FloatTensor))
            loss_plot.append(loss)
            # credit assignment
            loss.backward()
            # update model weights
            optimizer.step()
            running_loss += loss.item()
            
    train_loss=running_loss/len(train_dl)
    train_losses.append(train_loss)
    #Print the progress
    if (epoch+1) % 10 == 0:
        print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch +1, epochs, loss.item()))



In [ ]:
plt.title("Loss plot")
plt.xlabel("epoch")
plt.ylabel("loss")
plt.plot(train_losses,label="training loss")

In [ ]:
#Using the NN inside HMC to sample points
from surrogate import NeuralGrad_pytorch
grad_hat = NeuralGrad_pytorch(model_torch)
phi_NNg ,p_NNg_prob = HMC(Nmd=5,eps=1/5.,surrogate = grad_hat)
print("actualHMC_prob=",actualHMC_prob.mean())
print("NNg_prob=",p_NNg_prob.mean())

# Correlators

In [ ]:
#Exact co-relators data 
exactData = open("exactFiles/U4B6.dat").readlines()
exT = []
exBonding = []
exAntiBonding = []
exAA = []
exAB = []
exBA = []
exBB = []
for i in range(len(exactData)):
    split = exactData[i].split()
    exT.append(float(split[0]))   # tau
    exAntiBonding.append(float(split[1]))  # anti-bonding
    exBonding.append(float(split[2]))      # bonding
    exAA.append(float(split[3]))  # cAA
    exAB.append(float(split[4]))  # cAB
    exBA.append(float(split[5]))  # cBA
    exBB.append(float(split[6]))  # cBB

In [ ]:
#Calculating co-relators 

def correlators(sample):

    corrUp_b = [ [] for t in range(Nt)] # <===== each correlator has Nt elements
    corrUp_ab = [ [] for t in range(Nt)]
    Cxx = [ [] for t in range(Nt)]
    Cxy = [ [] for t in range(Nt)]
    Cyx = [ [] for t in range(Nt)]
    Cyy = [ [] for t in range(Nt)]

    nTherm = 200

    ### calculate the bonding/anti-bonding correlator
    for i in range(nTherm,nTrajs):
        # arrow up correlator
        phi = sample[i]
        Mphi(phi,expk,Nt)
        invMUp = np.linalg.inv(M)

        # we now construct the correlators
        # bonding correlator is .5*(Cxx+Cxy+Cyx+Cyy)
        # antibonding is .5*(Cxx-Cxy-Cyx+Cyy)
        for t in range(Nt):
            corrUp_b[t].append(np.real(.5*(invMUp[t*Nx+0][0*Nx+0]+invMUp[t*Nx+0][0*Nx+1]+
                                   invMUp[t*Nx+1][0*Nx+0]+invMUp[t*Nx+1][0*Nx+1])))
            corrUp_ab[t].append(np.real(.5*(invMUp[t*Nx+0][0*Nx+0]-invMUp[t*Nx+0][0*Nx+1]-
                                   invMUp[t*Nx+1][0*Nx+0]+invMUp[t*Nx+1][0*Nx+1])))
            Cxx[t].append(np.real(invMUp[t*Nx+0][0*Nx+0]))
            Cxy[t].append(np.real(invMUp[t*Nx+0][0*Nx+1]))
            Cyx[t].append(np.real(invMUp[t*Nx+1][0*Nx+0]))
            Cyy[t].append(np.real(invMUp[t*Nx+1][0*Nx+1]))

    # here I set up arrays that store the averages
    corrBond = []
    corrAntiBond = []

    # ok, I'm mixing up naming conventions.  Here 'A' = 'x' and 'B' = 'y'
    cAA = []
    cAB = []
    cBA = []
    cBB = []

    # now I calculate the averages for each timeslice
    for t in range(Nt):
        corrBond.append(np.mean(corrUp_b[t]))
        corrAntiBond.append(np.mean(corrUp_ab[t]))
        cAA.append(np.mean(Cxx[t]))
        cAB.append(np.mean(Cxy[t]))
        cBA.append(np.mean(Cyx[t]))
        cBB.append(np.mean(Cyy[t]))

    tau = np.linspace(0,beta-beta/Nt,Nt) # <===== this is the correct distancing for tau
    return corrBond,corrAntiBond,cAA,cAB,cBA,cBB,tau,corrUp_b,corrUp_ab,Cxx,Cxy,Cyx,Cyy

a_corrBond,a_corrAntiBond,a_cAA,a_cAB,a_cBA,a_cBB,a_tau,a_corrUp_b,a_corrUp_ab,a_Cxx,a_Cxy,a_Cyx,a_Cyy = correlators(actual_HMC)
corrBond,corrAntiBond,cAA,cAB,cBA,cBB,tau,corrUp_b,corrUp_ab,Cxx,Cxy,Cyx,Cyy = correlators(phi_NNg)



In [ ]:
#Binning and Bootstraping
def binAndBoot(corr,nBN,nBS,check):
    # corr is the array of of correlators 
    # nBN is the number or elements to bin
    # nBS is the number of bootstrap samples
    if check == 1:
        # making the correlators in correct form [samples]x[Nt]
        correlators = []
        for i in range(len(corr[0])): # range(samples)
            correlators.append([corr[t][i] for t in range(len(corr))]) #range(Nt)
    else:
        correlators = corr
        
    corrBN = [] #shape [(samples)/nBN x Nt]
    for i in range(int(len(correlators)/nBN)): #range(samples/nBN)
        corrBN.append(np.mean(correlators[i*nBN:(i+1)*nBN],axis=0))
        
    n = len(corrBN)  # length of ensemble (samples/nBN)
    bsMeans = []
    
    # here are my bootstrap indices
    bsSamples = [ np.random.randint(n,size=n) for bs in range(nBS)]  #shape [nBS x (samples/nBN) ]
    for sample in bsSamples: #shape [(samples/nBN) ]
        bsMeans.append(np.mean([corrBN[index] for index in sample],axis=0))   # mean along rows( [(samples/nBN) x Nt]) 
    # bsMeans.append([1 x Nt]), shape [nBS x 16]
    # returns array of shape[1 x Nt]
    return np.std(bsMeans,axis=0) 

In [ ]:
#plotting correlators

fig,ax = plt.subplots(2,2,figsize=(15,15))



axx = ax[0][0]
#axx.plot(a_tau,a_corrBond,'ro',label= r'$HMCBonding$')
axx.errorbar(a_tau,a_corrBond,yerr=binAndBoot(a_corrUp_b,100,100,1),marker='o',label='HMCBonding')

#axx.plot(a_tau,a_corrAntiBond,'bo', label = '$HMCAntibonding$')
axx.errorbar(a_tau,a_corrAntiBond,yerr=binAndBoot(a_corrUp_ab,100,100,1),marker='o',label='HMCAntiBonding')

axx.plot(exT,exBonding,'k--',label='exact')
axx.plot(exT,exAntiBonding,'k--')

axx.grid()
axx.set_xlabel(r'$t$',fontsize=18)
axx.set_yscale('log')
axx.legend(loc='best')

axx = ax[0][1]
#axx.plot(tau,corrBond,'go',label= r'$NNgBonding$')
axx.errorbar(tau,corrBond,yerr=binAndBoot(corrUp_b,100,100,1),marker='o',label='NNgHMCBonding')

#axx.plot(tau,corrAntiBond,'yo', label = '$NNgAntibonding$')
axx.errorbar(tau,corrAntiBond,yerr=binAndBoot(corrUp_ab,100,100,1),marker='o',label='NNgHMCAntiBonding')

axx.plot(exT,exBonding,'k--',label='exact')
axx.plot(exT,exAntiBonding,'k--')

axx.grid()
axx.set_xlabel(r'$t$',fontsize=18)
axx.set_yscale('log')
axx.legend(loc='best')

axx = ax[1][0]
axx.errorbar(a_tau,a_corrBond,yerr=binAndBoot(a_corrUp_b,100,100,1),marker='o',label='HMCBonding')
axx.errorbar(tau,corrAntiBond,yerr=binAndBoot(corrUp_ab,100,100,1),marker='o',label='NNgHMCAntiBonding')

axx.errorbar(a_tau,a_corrAntiBond,yerr=binAndBoot(a_corrUp_ab,100,100,1),marker='o',label='HMCAntiBonding')
axx.errorbar(tau,corrBond,yerr=binAndBoot(corrUp_b,100,100,1),marker='o',label='NNgHMCBonding')

axx.plot(exT,exBonding,'k--',label='exact')
axx.plot(exT,exAntiBonding,'k--')

axx.grid()
axx.set_xlabel(r'$t$',fontsize=18)
axx.set_yscale('log')
axx.legend(loc='best')

axx = ax[1][1]
axx.errorbar(exT[::50],exBonding[::50],yerr=binAndBoot(exBonding[::50],100,100,0),marker='o',label='exactBonding')
axx.errorbar(exT[::50],exAntiBonding[::50],yerr=binAndBoot(exAntiBonding[::50],100,100,0),marker='o',label='exactAntibonding')
axx.set_xlabel(r'$t$',fontsize=18)
axx.set_yscale('log')
axx.grid()
axx.legend(loc='best')

plt.suptitle('U={}, beta={}, Nt={}\nsamples={},prob_HMC={},prob_NNgHMC={}'
            .format(U,beta,Nt,nTrajs,actualHMC_prob.mean(),p_NNg_prob.mean()),fontsize=20)


#axx.plot(exT[::50],exBonding[::50],'ro',label='exactBonding')
#axx.plot(exT[::50],exAntiBonding[::50],'bo',label='exactAntibonding')
#axx.plot(np.array(a_corrBond) - np.array(corrBond),'ro',label= r'$diff_Bonding$')
#axx.plot(np.array(a_corrAntiBond) - np.array(corrAntiBond),'bo', label = '$diff_Antibonding$')

# plt.savefig("testM_U{}_B{}_10000.pdf".format(U,beta))

In [ ]:
fig,ax = plt.subplots(2,2,figsize=(15,10),sharex=True)

aax = ax[0][0]
aax.plot(exT,exAA,'k--',label='exact')

#aax.plot(tau,cAA,'b.',label='NNgMC')
aax.errorbar(tau,cAA,yerr=binAndBoot(Cxx,100,400,1),marker='o',label='NNgHMC')

#aax.plot(a_tau,a_cAA,'r.',label='HMC')
aax.errorbar(a_tau,a_cAA,yerr=binAndBoot(a_Cxx,100,400,1),marker='o',label='HMC')

aax.set_yscale('log')
aax.legend(loc='best')
aax.set_ylabel(r'$C_{AA}(\tau)$')
aax.grid()

aax = ax[0][1]
aax.plot(exT,exAB,'k--',label='exact')

#aax.plot(tau,cAB,'b.',label='NNgHMC')
aax.errorbar(tau,cAB,yerr=binAndBoot(Cxy,100,400,1),marker='o',label='NNgHMC')

#aax.plot(a_tau,a_cAB,'r.',label='HMC')
aax.errorbar(a_tau,a_cAB,yerr=binAndBoot(a_Cxy,100,400,1),marker='o',label='HMC')


aax.set_ylabel(r'$C_{AB}(\tau)$')
aax.legend(loc='best')
aax.grid()

aax = ax[1][0]
aax.plot(exT,exBA,'k--',label='exact')

#aax.plot(tau,cBA,'b.',label='NNgHMC')
aax.errorbar(tau,cBA,yerr=binAndBoot(Cyx,100,400,1),marker='o',label='NNgHMC')

#aax.plot(a_tau,a_cBA,'r.',label='HMC')
aax.errorbar(a_tau,a_cBA,yerr=binAndBoot(a_Cyx,100,400,1),marker='o',label='HMC')


aax.grid()
aax.set_xlabel(r'$\tau$')
aax.set_ylabel(r'$C_{BA}(\tau)$')
aax.legend(loc='best')

aax = ax[1][1]
aax.plot(exT,exBB,'k--',label='exact')

#aax.plot(tau,cBB,'b.',label='NNgHMC')
aax.errorbar(tau,cBB,yerr=binAndBoot(Cyy,100,400,1),marker='o',label='NNgHMC')

#aax.plot(a_tau,a_cBB,'r.',label='HMC')
aax.errorbar(a_tau,a_cBB,yerr=binAndBoot(a_Cyy,100,400,1),marker='o',label='HMC')

aax.grid()
aax.set_yscale('log')
aax.set_xlabel(r'$\tau$')
aax.set_ylabel(r'$C_{BB}(\tau)$')
aax.legend(loc='best')

plt.suptitle('U={}, beta={}, Nt={}\nsamples={},prob_HMC={},prob_NNgHMC={}'
            .format(U,beta,Nt,nTrajs,actualHMC_prob.mean(),p_NNg_prob.mean()),fontsize=20)

# plt.savefig("testM_correlator_logerror_U{}_B{}_10000.pdf".format(U,beta))